In [2]:
from tests.test_utils import FakeCustomerChain

from app.main import main_flow

from typing import List
import time

from app.system import Conversation, Task, Step, System
from app.task_resolver import GatherBookingInfoResolver, HouseSelectionResolver, GatherUserInfoResolver, BookingConfirmationResolver
# from task_resolver.tasks import create_make_reservation_task, create_task_router_task
# import langchain

def main_flow(message: str, customer_number: str, system: System): 

    if message == "exit":
        system.save_conversation(Conversation(customer_number, create_task_router_task()))
        return "Conversacion reiniciada. Puedes comenzar de nuevo!"

    conversation = system.get_conversation(customer_number)
    conversation.add_user_message(message)
    logger.debug(f"Conversation: {conversation.get_messages()}")
    task_result = conversation.task.run(conversation.get_messages())
    if conversation.task.name == "TASK_ROUTER_TASK":
        if task_result["task_id"] != "OTHER":
            # Here I know already that he wants to make a reservation.
            task = create_task(task_result["task_id"])
            conversation.task = task
            response = task.run(conversation.get_messages())
            # if not task.steps[-1].reply_when_done:
            #     return main_flow()                
            conversation.add_assistant_message(response)
            system.save_conversation(conversation)
            return response
        else:
            conversation.add_assistant_message(task_result["text"])
            system.save_conversation(conversation)
            return task_result["text"]
    else:
        conversation.add_assistant_message(task_result)
        system.save_conversation(conversation)
        return task_result

def test_conversation(customer: FakeCustomerChain, first_msg="Hola") -> List[str]:

    system = System()
    customer_number="99513718"

    chat_history = [first_msg]

    for i in range(10):
        bnbot_response = main_flow(first_msg, customer_number)
        chat_history.append(bnbot_response)
        customer_response = customer(chat_history)
        chat_history.append(customer_response)
    
    return chat_history

ModuleNotFoundError: No module named 'system'

In [ ]:




# def test_task(message: str, task: Task): 
#     conversation = system.get_conversation(customer_number)
#     conversation.add_user_message(message)
#     print(f"Conversation: {conversation.get_messages()}")
#     response = task.run(conversation.get_messages())
#     conversation.add_assistant_message(response)
#     system.save_conversation(conversation)
#     return response